In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import xlsxwriter
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill
import tkinter as tk
import customtkinter as ctk
import threading
import time
# pip install beautifulsoup
# pip install requests
# pip install pandas
# pip install xlsxwriter

In [ ]:
base_url="https://kontakt.az"
response = requests.get(base_url)
productList = list()

In [ ]:
def scrape_products(searchValue, stopEvent):
    page=1
    while page < 10:
        url = base_url + "/search/?p=" + str(page) + "&q=" + searchValue
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        productCards = soup.findAll("div", attrs={"class":"prodItem product-item"})
        for productCard in productCards:
                sku = productCard.get("data-sku")
                # if "TM-DG-SBP-1105-SM" in sku:
                    
                productId = productCard.get("id")
                productName = productCard.find("div", attrs={"class":"prodItem__title"}).text
                productPriceBox = productCard.find("div", attrs={"class":"prodItem__prices prodItem__prices--active"})
                productImage = productCard.find("img", attrs={"class":"product-image"}).get("src")
                originalPrice = productPriceBox.find("i")
                discountedPrice = productPriceBox.find("b")
                interestFreeTerm = productCard.find("span")
                print(f"name: {productName}\nprice: {originalPrice.text}\ndiscounted price: {discountedPrice.text}\nimage link: {productImage}\ninterestFreeTerm: {interestFreeTerm}" + "\n")
                productList.append([productId, productName, originalPrice, discountedPrice, productImage])
        page+=1
        print(page)


In [79]:
def on_search():
    global search_thread
    searchValue = entry1.get()
    stopEvent.clear()
    if searchValue:
        search_thread = threading.Thread(target=fetch_products, args=(searchValue, stopEvent))
        search_thread.start()

def on_cancel():
    stopEvent.set()
    print("Canceling the process...")

app = ctk.CTk()
app.geometry("400x250")
app.title("Search Interface")

entry_label = ctk.CTkLabel(app, text="Search Value:")
entry_label.pack(pady=0)
entry1 = ctk.CTkEntry(app, width=300)
entry1.insert(0, "telefon")
entry1.pack(pady=10)

entry_label = ctk.CTkLabel(app, text="Data Sku Value:")
entry_label.pack(pady=0)
entry2 = ctk.CTkEntry(app, width=300)
entry2.insert(0, "TM-DG-SBP-1105-SM")
entry2.pack(pady=10)

# Search and Cancel Buttons
search_button = ctk.CTkButton(app, text="Search", command=on_search)
search_button.grid(row=2, column=0, padx=20, pady=(0, 20), sticky="w")

cancel_button = ctk.CTkButton(app, text="Cancel", command=on_cancel)
cancel_button.grid(row=2, column=1, padx=20, pady=(0, 20), sticky="w")


stopEvent = threading.Event()

app.mainloop()


TclError: cannot use geometry manager grid inside . which already has slaves managed by pack

In [ ]:
df = pd.DataFrame(productList, columns=["ID", "Məhsulun adı", "Məhsulun endirimsiz qiyməti", "Məhsulun nağd alışa endirimli qiyməti", "Məhsulun şəkil linki"])

In [ ]:
# Bütün datanın excel faylına extract olunması
# Exrteacting all data to excel file
with pd.ExcelWriter("dataframe.xlsx", engine="xlsxwriter") as dataframe:
    df.to_excel(dataframe, sheet_name="sheet1", index=False)
    workbook = dataframe.book
    worksheet = dataframe.sheets["sheet1"]

    for idx, col in enumerate(df.columns):
        # cədvəldəki ən uzun data eninin tapılması
        max_length = max(df[col].astype(str).map(len).max(), len(col)) + 2  # 2 padding dəyəri kimidi
        worksheet.set_column(idx, idx, max_length)  # həmin uzunluğun header uzunluğu kimi təyini

    # header format
    header_format = workbook.add_format({
        'align': 'left',
        'bg_color': '#59788E',
        'italic': True
    })
    for col_num, value in enumerate(df.columns.values):
        worksheet.write(0, col_num, value, header_format)